In [ ]:
import tensorflow as tf
import argparse
import numpy as np
import sys
sys.path.append("/data/slim/models/research/slim/")
from nets import nets_factory
import matplotlib.pyplot as plt
%matplotlib inline  

sys.path.append("../")
from src.data_preparation import dataset

slim = tf.contrib.slim
PRETAIN_MODEL = "inception_v1"
num_classes = 120
WIDTH = 224
INPUT_WIDTH= 224
CHANNEL = 3

TRAIN_TF_RECORDS = "/data/dog_breeds/stanford_ds/train/train.tfrecord"
TRAIN_BATCH_SIZE = 10

In [ ]:
with tf.Graph().as_default(), tf.Session() as sess:
    # First create the dataset and load one batch
    images_raw, images, labels = dataset.load_batch(
        TRAIN_TF_RECORDS,
        TRAIN_BATCH_SIZE,
        INPUT_WIDTH,
        INPUT_WIDTH,
        is_training=False,
        num_epochs=1)
#     images = tf.cast(images * 255, tf.uint8)
    sess.run(tf.local_variables_initializer())
    
    with slim.queues.QueueRunners(sess):
        image_raw_batch, image_batch = sess.run([images_raw, images])
        print(image_batch.max(), image_batch.min())

#     print(image_batch[0])   
    image_precessed_batch = (image_batch * 127 + 127).astype(np.uint8)
#     print(image_precessed_batch[0])
    print(image_precessed_batch.max(), image_precessed_batch.min())
    plt.imshow(image_precessed_batch[0])
    plt.show()

    plt.imshow(image_raw_batch[0, :].astype(np.uint8))
    plt.show()


In [ ]:
with tf.Graph().as_default(), tf.Session() as sess:
    x_input = tf.placeholder(tf.float32, [None, WIDTH, HEIGHT, CHANNEL])
    label = tf.placeholder(tf.int64)
    
    net_fn = nets_factory.get_network_fn(
            PRETAIN_MODEL,
            num_classes,
            is_training=False)
    
    logits, end_points = net_fn(x_input)
    predictions = end_points['Predictions']
    
    sess.run(tf.global_variables_initializer())

    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=label, logits=logits)
    
    x = np.ones([1, HEIGHT, WIDTH, CHANNEL])
    loss_, logits_ = sess.run([loss, logits], {x_input: x, label: [1]})
    print("{}, {}".format(loss_, logits_))